<a href="https://colab.research.google.com/github/ayushkalani/logic-dataset-creation-nlp/blob/master/NLPDatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!python --version

Mounted at /content/drive
Python 3.7.12


In [ ]:
# Wordnet
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
NOUNS = [word for synset in wn.all_synsets('n') for word in synset.lemma_names()]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
!pip install names

In [ ]:
relations = ['RelatedTo','IsA','FormOf','PartOf','SimilarTo','HasA','UsedFor','CapableOf','AtLocation','Causes','HasSubevent','HasPrerequisite','HasProperty','MotivatedByGoal','ObstructedBy','Desires','CreatedBy','Synonym','DistinctFrom','DerivedFrom','SymbolOf','DefinedAs','MannerOf','LocatedNear','HasContext','EtymologicallyRelatedTo','EtymologicallyDerivedFrom','CausesDesire','MadeOf','ReceivesAction']

In [ ]:
import names

def create_premise(klass, sent_1, sent_2):
  premise = ""
  premise += (sent_1 + ". " + sent_2)
  name = names.get_first_name()
  inference = "{} is a {}".format(name, klass)
  premise += (". "+ inference)
  return [premise, name]

def get_qa(questions, ans):
  return {"Questions": questions, "Answers": [ans] * len(questions) }

def create_question_type_1(questionable_name, word):
  return "Does {} has {}?".format(questionable_name, word)

def create_question_type_2(questionable_name, word):
  return "Does {} not has {}?".format(questionable_name, word)

def create_question_type_3(questionable_name, word_1, word_2, operator):
  return "Does {} has {} {} {}?".format(questionable_name, word_1, operator, word_2)

# does john has tail?
def create_question_type_4(questionable_name, word):
   antonym = get_antonym_of_word(word)
   if antonym:
      return "Does {} has {}?".format(questionable_name, antonym)
   else:
     return None

def get_antonym_of_word(klass):
  url = 'http://api.conceptnet.io/query?end=/c/en/{}&rel=/r/Antonym&limit=1000&filter=/c/en'.format(klass)
  obj = requests.get(url).json()
  # return the first antonym
  for edge in obj['edges']:
    return (edge['start']['label'] or '')

In [ ]:
# Conjunction
import requests, re, traceback
import pandas as pd
pattern = r"[\([{})\]]"
OPERATORS = ["and", "or"]
YES ="yes"
NO = "no"

def mapping_for_class(klass):
  url = 'http://api.conceptnet.io/query?start=/c/en/{}&rel=/r/HasA&limit=1000'.format(klass)
  obj = requests.get(url).json()
  dataum = {}
  df_klass = pd.DataFrame()
  for edge in obj['edges']:
    dataum[edge['end']['label']] = re.sub(pattern, "", edge['surfaceText'])
  
  temp = list(dataum)
  klass_examples = {}
  result = {'Questions': [], 'Answers': []}
  for i in range(0, len(temp)):
    word_1 = temp[i]
    for j in range(i+1, len(temp)):
      try:
          #word_2 = temp[temp.index(word_1) + 1]
          word_2 = temp[j]
          sent_1 = dataum[word_1]
          sent_2 = dataum[word_2]
          premise, questionable_name = create_premise(klass, sent_1, sent_2)
          questions_type1 = []
          questions_type2 = []
          questions_type3 = []
          questions_type4 = []
          # does john has tail?
          for w in [word_1, word_2]:
            questions_type1.append(create_question_type_1(questionable_name, w))
            questions_type2.append(create_question_type_2(questionable_name, w))
            questions_type4.append(create_question_type_4(questionable_name, w))
          # does johhn has tail or fur?
          # does johhn has tail and fur?
          for oprand in OPERATORS:
            questions_type2.append(create_question_type_3(questionable_name, word_1, word_2, oprand))
          questions_type4_final_list = list(filter(None, questions_type4))
          q1=get_qa(questions_type1, YES)
          q2=get_qa(questions_type2, NO)
          q3=get_qa(questions_type3, YES)
          q4=get_qa(questions_type4_final_list, NO)
          result['Rules'] = 'Conjunction'
          result['Context'] = premise
          a = q1['Questions'] + q2['Questions'] + q3['Questions'] + q4['Questions']
          result['Questions'].extend(a)
          b = q1['Answers'] + q2['Answers'] + q3['Answers'] + q4['Answers']
          result['Answers'].extend(b)
          df_klass = pd.concat([df_klass,pd.DataFrame(result)],ignore_index=True)
      except Exception as e:
        print(e)
        print(traceback.format_exc())
        return df_klass
      except (ValueError, IndexError):
        print(traceback.format_exc())
        return df_klass
  return df_klass

In [ ]:
mapping_for_class("animals")

In [ ]:
from uuid import uuid4
df = pd.DataFrame(columns=['UUID','Rules','Context','Questions','Answers'])
current_words = ["animals", "countries", "cities", "sports", "cultures", "dogs", "cats", "buildings", "colleges", "subjects", "lakes", "mountains", "social", "dishes"]

for noun in current_words:
  try:
    dn = mapping_for_class(noun)
    df = pd.concat([df, pd.DataFrame(dn)], ignore_index=True)
  except Exception:
    continue
    #print(traceback.format_exc())

# for noun in NOUNS[100:200]:
#   dn = mapping_for_class(noun)
#   df = pd.concat([df, pd.DataFrame(dn)], ignore_index=True)

df['UUID'] = df.index.to_series().map(lambda x: uuid4())
df.to_csv("Dataset5_AyushKalani.csv", sep=',', index=False)

In [ ]:
len(df)

NameError: ignored